## Data preparation

This dataset contains ESC final results from 1956 to 2019.

This notebook cleans the data and replaces categorical values with numeric ones.

In [1]:
import pandas as pd
import numpy as np

Read in data from csv file

In [2]:
data = pd.read_csv("../data/ESCDB2.csv", sep=";", encoding="UTF-8")

# Removing potential unneccesary spaces from dataframe columns
for column in data.select_dtypes(include='object'):
    data[column] = data[column].str.strip()

# Dropping duplicate column "songid"
data.drop('songid', axis=1)

,song_id,Year,Order,Country,Singer,Title,Points,Place,in_english,lovementions,population
0,1,1956,1,Netherlands,Jetty Paerl,De Vogels Van Holland,0.0,0.0,False,0.0,17000000.0
1,2,1956,2,Switzerland,Lys Assia,Das Alte Karussel,0.0,0.0,False,0.0,8420000.0
2,3,1956,3,Belgium,Fud Leclerc,Messieurs Les Noyés Dans La Seine,0.0,0.0,False,4.0,11350000.0
3,4,1956,4,Germany,Walter Andreas Schwarz,Das Lied Vom Großen Glück,0.0,0.0,False,0.0,82790000.0
4,5,1956,5,France,Mathé Altéry,Le Temps Perdu,0.0,0.0,False,6.0,67120000.0
...,...,...,...,...,...,...,...,...,...,...,...
1312,1329,2019,33,Poland,Tulia,Fire of Love,NaN,NaN,False,37970000.0,NaN
1313,1331,2019,35,Romania,Ester Peony,On a Sunday,NaN,NaN,True,19640000.0,NaN
1314,1333,2019,37,Russia,Sergey Lazarev,Scream,NaN,NaN,True,144500000.0,NaN
1315,1335,2019,39,Slovenia,Zala Kralj & Gašper Šantl,Sebi,NaN,NaN,False,1990000.0,NaN


Making country column numeric

In [3]:
countries = data["Country"].unique()
countries.sort()
countries

countries_to_numbers = {}

for i in range(len(countries)):
    countries_to_numbers[countries[i]] = i

data["Country"] = data["Country"].map(countries_to_numbers)

Making the singer column numeric

This will be a boolean value:
* **0** if the singer has only performed once in ESC
* **1** if the singer has performed multiple times in ESC

In [4]:
data["Singer"] = data["Singer"].map(lambda x: 0 if data["Singer"].value_counts()[x] == 1 else 1)

Making title column numeric.

We are extracting two attributes:
* 1 if song title included word "Love" 0 otherwise
* 1 if song title is more than 1 word, 0 otherwise 

In [5]:
data["love_in_title"] = data["Title"].map(lambda x: 1 if 'love' in x.lower() else 0)

In [6]:
data["Title"] = data["Title"].map(lambda x: len(x.split()))
data = data.rename(columns={"Title" : "title_word_count"})

Making "in_english" column numeric.

In [7]:
data["in_english"] = data["in_english"].map(lambda x: 1 if x else 0)

In [8]:
# display out the result
data

,song_id,Year,Order,Country,Singer,title_word_count,Points,Place,in_english,songid,lovementions,population,love_in_title
0,1,1956,1,33,0,4,0.0,0.0,0,1.0,0.0,17000000.0,0
1,2,1956,2,46,1,3,0.0,0.0,0,2.0,0.0,8420000.0,0
2,3,1956,3,6,1,6,0.0,0.0,0,3.0,4.0,11350000.0,0
3,4,1956,4,17,0,5,0.0,0.0,0,4.0,0.0,82790000.0,0
4,5,1956,5,15,0,3,0.0,0.0,0,5.0,6.0,67120000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,1329,2019,33,35,0,3,NaN,NaN,0,NaN,37970000.0,NaN,1
1313,1331,2019,35,37,0,3,NaN,NaN,1,NaN,19640000.0,NaN,0
1314,1333,2019,37,38,1,1,NaN,NaN,1,NaN,144500000.0,NaN,0
1315,1335,2019,39,43,0,1,NaN,NaN,0,NaN,1990000.0,NaN,0
